# Web-Summary Notebook — LM Studio
_A tiny utility to fetch a page → extract text → ask the model for a short summary._

### Imports

In [1]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

### OpenAI-compatible client

In [ ]:
# Configure OpenAI-compatible client (GLM endpoint)
client = OpenAI(
  base_url="http://localhost:1234/v1",
)

In [ ]:
# quick smoke test (optional)
message = "Hello, GLM! This is my first ever message to you! Hi!"
response = client.chat.completions.create(model="llama-3.2-3b-instruct", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)


Hello! A very warm welcome to you!

It's a genuine pleasure to meet you, and I'm so excited for our first conversation together. Thank you for reaching out!

I'm here to help with just about anything – from answering questions and brainstorming ideas to creative writing or just a friendly chat.

So, what's on your mind? How can I help you today?


### Web fetcher

In [5]:
# Some websites need proper headers when fetching them
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """Create this Website object from the given url using BeautifulSoup."""
        self.url = url
        response = requests.get(url, headers=headers, timeout=20)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        # strip junk
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True) if soup.body else ""

In [7]:
rp = Website("https://rp30.us")
print(rp.title)
print(rp.text)

Rohith's Portfolio
RP.
Home
Skills
Experience
Education
Projects
Contact
Home
Skills
Experience
Education
Projects
Contact
Rohith Goud Panjala
Full-Stack Software Engineer & Problem Solver
Engineering Scalable & Impactful Solutions
I'm a Full-Stack Software Engineer who builds fast, scalable, and intelligent systems that solve real-world problems. I thrive on turning complexity into clarity by designing solutions that are clean, efficient, and built to last. With a sharp eye for detail and a passion for technology, I bring energy and ownership to every project I touch. I don't just code. I deliver outcomes, move fast, and grow with every challenge.
Resume
Get In Touch
Scroll
My Skills
Technical expertise and tools I work with
Scroll
Work Experience
My professional journey and key contributions
Jun 2024 - Aug 2024
Software Engineer Intern
ZROin
Remote (Baltimore, MD)
Engineered scalable backend services in Nest.js, enhancing ZROin's platform for 50+ small businesses by integrating 3D st

### Prompt builders

In [8]:
# system prompt — tweak freely
system_prompt = (
    "You are an assistant that analyzes the contents of a website "
    "and provides a short summary, ignoring navigation text. "
    "Respond in markdown."
)

def user_prompt_for(website: Website):
    up = f"You are looking at a website titled {website.title}\n"
    up += "The contents of this website is as follows:\n"
    up += "Please provide a short summary of this website in markdown.\n"
    up += "If it includes news or announcements, summarize these too.\n\n"
    up += website.text
    return up

def messages_for(website: Website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [9]:
print(user_prompt_for(rp))

You are looking at a website titled Rohith's Portfolio
The contents of this website is as follows:
Please provide a short summary of this website in markdown.
If it includes news or announcements, summarize these too.

RP.
Home
Skills
Experience
Education
Projects
Contact
Home
Skills
Experience
Education
Projects
Contact
Rohith Goud Panjala
Full-Stack Software Engineer & Problem Solver
Engineering Scalable & Impactful Solutions
I'm a Full-Stack Software Engineer who builds fast, scalable, and intelligent systems that solve real-world problems. I thrive on turning complexity into clarity by designing solutions that are clean, efficient, and built to last. With a sharp eye for detail and a passion for technology, I bring energy and ownership to every project I touch. I don't just code. I deliver outcomes, move fast, and grow with every challenge.
Resume
Get In Touch
Scroll
My Skills
Technical expertise and tools I work with
Scroll
Work Experience
My professional journey and key contribut

## My Runs
List any sites you want summarized below and run the next cell.

### Model call + display helpers

In [ ]:
def summarize(url):
    website = Website(url)
    response = client.chat.completions.create(
        model = "GLM-4.6",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [11]:
summarize("https://rp30.us")

"\n# Rohith Goud Panjala's Portfolio Summary\n\nThis is the personal portfolio for Rohith Goud Panjala, a Full-Stack Software Engineer.\n\n## About\nRohith is a software engineer who builds fast, scalable, and intelligent systems to solve real-world problems. He focuses on delivering clean, efficient, and lasting solutions.\n\n## Experience\nRohith has experience as a Software Engineer Intern at **ZROin**, where he engineered backend services for an e-commerce platform, and as a Backend Developer Intern at **Reel Talk**, developing real-time backends for multiple platforms. He has also worked as a Research Assistant at **The Ethical Software Lab at UMBC** and as a Programmer Analyst at **Cognizant**, where he built a full-stack application for clinical workflows and improved performance by 30%.\n\n## Education\n*   **Master of Professional Studies in Software Engineering** from the University of Maryland Baltimore County (UMBC), GPA 3.87/4.0.\n*   **Bachelor of Technology in Electronic

In [12]:
# A function to display this nicely in the Jupyter output, using markdown
display_summary("https://rp30.us")


This is the personal portfolio website for Rohith Goud Panjala, a Full-Stack Software Engineer.

### Summary

Rohith presents himself as a problem-solver who engineers fast, scalable, and intelligent systems. The website showcases his professional experience, education, and key projects to demonstrate his technical expertise and passion for delivering impactful solutions.

### Experience

His professional background includes roles as a Software Engineer Intern at **ZROin** and Backend Developer Intern at **Reel Talk**, where he built scalable backend services and real-time systems. Previously, he was a Programmer Analyst at **Cognizant**, developing full-stack applications for clinical workflows, and a Research Assistant at **The Ethical Software Lab, UMBC**, contributing to ethical software research.

### Projects

A highlighted project is **CRWLR**, a full-stack cloud-native platform that uses AI and NLP to crawl, summarize, and analyze legal documents like Terms of Service. The project details his use of technologies like Next.js, FastAPI, the Gemini API, and a custom crawler, demonstrating his end-to-end development capabilities.

### Education

Rohith is currently pursuing a Master of Professional Studies in Software Engineering at the **University of Maryland Baltimore County (UMBC)** with a 3.87 GPA. He also holds a Bachelor of Technology in Electronics and Communication Engineering.

### Announcement

Rohith explicitly states that he is **currently open to new opportunities** and invites potential employers to connect with him via email or LinkedIn.

### Let's try more websites

In [13]:
display_summary("https://cnn.com")

# CNN Breaking News Summary

The website is the homepage for CNN, featuring a wide array of current news, politics, business, health, entertainment, and sports stories.

## Key News & Announcements

*   **Gaza Ceasefire Talks:** Live updates cover ceasefire negotiations set to begin in Egypt, with President Trump pressuring both Israel and Hamas to strike a deal. An exclusive report details Trump's warning of "obliteration" to Hamas if it does not cede power.
*   **US Politics & Government:**
    *   A judge has blocked the deployment of National Guard troops to Portland, while Trump has simultaneously authorized troops for Chicago.
    *   The government shutdown has entered its second week, with a lack of "serious" bipartisan talks.
    *   Trump is considering a massive bailout for American farmers hurt by his trade war.
    *   Trump has called on Google to add Univision back to YouTube TV.
*   **World News:**
    *   Hundreds of trekkers are still awaiting rescue on Mount Everest in Tibet after a blizzard.
    *   Five people were killed in a Russian overnight air attack on Ukraine.
*   **US News:**
    *   Gunmen killed two people and injured twelve in a downtown shootout in Montgomery, Alabama.
    *   Ex-NFL quarterback Mark Sanchez was hospitalized and arrested following an apparent stabbing.
*   **Business:**
    *   Rite Aid is closing all its remaining stores after 63 years in business.
    *   The US Treasurer has confirmed a draft for a $1 Trump coin is "real."
*   **Entertainment:**
    *   Coverage of Bad Bunny's appearance on 'SNL' and Taylor Swift's new album.
*   **Sports:**
    *   Mexico soccer star Omar Bravo has been arrested on suspicion of child sexual abuse.
    *   Coverage of the NFL, WNBA Finals, and college football.

In [14]:
display_summary("https://anthropic.com")

# Anthropic: AI Research and Safety-Focused Products

Anthropic is a public benefit corporation dedicated to building safe AI systems that benefit humanity. The company emphasizes responsible AI development through research, policy work, and product design.

## Key Products
- **Claude**: A family of AI models (Opus, Sonnet, Haiku)
- **Claude Code**: AI development tool
- **Claude Developer Platform**: For building AI applications

## Recent Announcements
- **Claude Sonnet 4.5**: New model optimized for agents, coding, and computer use
- **Claude Sonnet 4 with 1M context**: Enhanced context window
- **Claude Opus 4.1**: Latest version of their most capable model

## Featured Research & Initiatives
- Anthropic Economic Index (studying societal impacts)
- Project Vend (policy initiative)
- Research on agentic misalignment and AI alignment
- Interpretability work on understanding LLM thought processes

## Safety Focus
The company maintains a Responsible Scaling Policy and offers educational resources through the Anthropic Academy. They position themselves as leaders in AI safety research and responsible development practices.

In [ ]:
display_summary("https://openai.com")